<a href="https://colab.research.google.com/github/BHouwens/DL_From_Foundations/blob/main/02_fully-connected.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Forward and Backward Passes**

In this notebook we'll be looking at setting up the forward and backward passes for the fully connected model architecture.

## **Colab Setup**
The setup structure for this will depend on the environment. I'm assuming a Google Colab environment in this case, which will require the following setup from the Github repo, and assumes that the repo has already been cloned into Google Drive:

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

%cd gdrive/My Drive/git_folder/DL_From_Foundations
! git pull

Mounted at /content/gdrive
/content/gdrive/My Drive/git_folder/DL_From_Foundations
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), done.
From https://github.com/BHouwens/DL_From_Foundations
   dca6ac7..7c424fc  main       -> origin/main
Updating dca6ac7..7c424fc
Fast-forward
 02_fully-connected.ipynb | 179 +++++++++++++++++++++++++++++++++++++----------
 1 file changed, 142 insertions(+), 37 deletions(-)


In [2]:
%load_ext autoreload
%autoreload 2

%matplotlib inline



---



## **Normalize Data**

In [57]:
#export
from exp.nb_01 import *
from keras.datasets import mnist

def get_data():
    # Load the data into the train and validation sets
    (x_train, y_train), (x_valid, y_valid) = mnist.load_data()

    # Map the sets to tensors
    x_train,y_train,x_valid,y_valid = map(tensor, (x_train,y_train,x_valid,y_valid))

    # Flatten the 28 * 28 to match the course
    x_train = torch.flatten(x_train, 1)
    x_valid = torch.flatten(x_valid, 1)

    return x_train, x_valid, y_train, y_valid

def normalize(x, m, s): return (x-m)/s

In [60]:
x_train, x_valid, y_train, y_valid = get_data()

print(y_train.shape)
print(y_valid.shape)

train_mean,train_std = x_train.float().mean(),x_train.float().std()
train_mean,train_std

x train shape (60000, 28, 28)
y train shape torch.Size([60000])
torch.Size([60000])
torch.Size([10000])


(tensor(33.3184), tensor(78.5675))

In [61]:
x_train = normalize(x_train, train_mean, train_std)

# NB: Use training, not validation mean for validation set
x_valid = normalize(x_valid, train_mean, train_std)

In [62]:
train_mean,train_std = x_train.mean(),x_train.std()
train_mean,train_std

(tensor(1.8892e-08), tensor(1.))

In [63]:
# Let's check out the shapes
n,m = x_train.shape
c = y_train.max()+1
n,m,c

(60000, 784, tensor(10, dtype=torch.uint8))

In [64]:
x_valid.shape

torch.Size([10000, 784])

Now we're talking!



---



## **Foundations (v1.0)**

In [9]:
# num hidden nodes
nh = 50

The aim of weight initialization is to prevent layer activation outputs from exploding or vanishing during the course of a forward pass through a deep neural network. If either occurs, loss gradients will either be too large or too small to flow backwards beneficially, and the network will take longer to converge, if it is even able to do so at all (think of what this looks like in a geometric sense, with gradients along a curve).

We can read more about weight initialisation specifically [here](https://towardsdatascience.com/weight-initialization-in-neural-networks-a-journey-from-the-basics-to-kaiming-954fb9b47c79).

In [10]:
# 2 layers, so we need 2 weights and 2 biases (kaiming init)
w1 = torch.randn(m,nh)/math.sqrt(m)
b1 = torch.zeros(nh)
w2 = torch.randn(nh,1)/math.sqrt(nh)
b2 = torch.zeros(1)

What's happening above is a bit of a cheat on regular Kaiming initialisation, since `torch.randn` returns a uniformly distributed random set. Kaiming is a modification of Xavier initialisation, which sets a layer's input weights to values from a uniformly random distribution bounded between 

$$\frac{\sqrt{6}}{\sqrt{n_i + n_{i+1}}}$$

where $n_i$ is the number of incoming network connections, or “fan-in,” to the layer, and $n_{i + 1}$ is the number of outgoing network connections from that layer, also known as the “fan-out.”

In [11]:
def linear_layer(x, w, b):
  """
  Creates a linear layer with a matrix multiplication of 
  x and w, plus the bias
  """
  return x@w + b

In [12]:
layer_activation = linear_layer(x_train, w1, b1)

Because we used Kaiming initialisation for our weights and biases we should also get a normalized mean and $\sigma$

In [13]:
# we used kaiming init, which is designed to do this
layer_activation.mean(), layer_activation.std()

(tensor(-0.0479), tensor(1.0176))

Now let's define ReLU for our activation:

In [14]:
# clamp_min replaces negatives with zeros
def relu(x): return x.clamp_min(0.)

But think about this geometrically. Let's say we plotted all our activations on the $x$ and $y$ axes. We know the activations already have a mean of 0 and a $\sigma$ of 1, but now ReLU went and replaced all the values in the negative portions of the $x$ and $y$ axes with 0 (so it "pulled" them toward the origin). 

We can't reasonably expect that the mean and $\sigma$ will still be the normalised values we want, can we?

In [15]:
layer_activation = relu(linear_layer(x_train, w1, b1))

# but now we don't have the right mean and stdev
layer_activation.mean(), layer_activation.std()

(tensor(0.3764), tensor(0.5813))

No dice!

From pytorch docs: `a: the negative slope of the rectifier used after this layer (0 for ReLU by default)`

$$\text{std} = \sqrt{\frac{2}{(1 + a^2) \times \text{fan_in}}}$$

This was introduced in the paper that described the Imagenet-winning approach from *He et al*: [Delving Deep into Rectifiers](https://arxiv.org/abs/1502.01852), which was also the first paper that claimed "super-human performance" on Imagenet (and, most importantly, it introduced resnets!).

The interesting insight the team made in this paper, regarding normalisation in particular, was to replace the 1 in the numerator with a 2!

In [16]:
# We can just change the weight initialisation from before by "adding a 2"
w1 = torch.randn(m,nh)*math.sqrt(2/m)

w1.mean(), w1.std()

(tensor(-0.0003), tensor(0.0505))

In [17]:
# Let's try again (although these runs can be random)
layer_activation = relu(linear_layer(x_train, w1, b1))

layer_activation.mean(), layer_activation.std()

(tensor(0.5150), tensor(0.7890))

That's closer, although not quite at the 0 mean and 1 $\sigma$ we're looking for. We can replace our little process with Pytorch's built in Kaiming initialiser and, just to make sure we're not cheating, we'll run the same params again and see if we get the same result:

In [18]:
from torch.nn import init

In [19]:
w1 = torch.zeros(m,nh)
init.kaiming_normal_(w1, mode='fan_out')

layer_activation = relu(linear_layer(x_train, w1, b1))
layer_activation.mean(), layer_activation.std()

(tensor(0.5144), tensor(0.8073))

Great! But what does `mode='fan_out'` really mean? According to Pytorch docs, `fan_out` ensures that the variance is kept consistent in the backward pass of the net. `fan_in`, by contrast, is used to ensure a consistent variance on the forward pass.

That's kind of strange though, because shouldn't we be doing the forward pass (or `fan_in`) first? Let's take a look at Pytorch's linear layer.

In [ ]:
torch.nn.functional.linear??

It turns out that Pytorch's linear layer first performs a transpose on the weights before actually doing the forward pass! Since our linear layer doesn't do any transposition we use `fan_out` instead.

In [20]:
# what if...?
def relu(x): return x.clamp_min(0.) - 0.5

This seems a bit hacky. We're subtracting a half from each entry in the input `x` after clamp, which seems like it would bring us closer to our desired mean and $\sigma$. A weird, patchy tweak but it's the kind of tweak that many papers try in order to discover or attempt something novel.

Let's see if our tweak works:

In [23]:
# kaiming init / he init for relu
w1 = torch.randn(m,nh)*math.sqrt(2./m )
t1 = relu(linear_layer(x_train, w1, b1))
t1.mean(),t1.std()

(tensor(0.1199), tensor(0.8908))

Now we can try to construct a basic model with our new utility functions.

In [26]:
def model(xb):
    l1 = linear_layer(xb, w1, b1)
    l2 = relu(l1)
    l3 = linear_layer(l2, w2, b2)
    return l3

In [27]:
%timeit -n 10 _=model(x_train)

10 loops, best of 5: 325 ms per loop


In [28]:
assert model(x_train).shape==torch.Size([x_train.shape[0],1])

### **Loss Function: MSE**



In [29]:
model(x_train).shape

torch.Size([60000, 1])

We need `squeeze()` to get rid of that trailing `(,1)`, in order to use MSE (Of course, MSE is not a suitable loss function for multi-class classification; we'll use a better loss function soon. We'll use MSE for now to keep things simple).

In [30]:
#export
def mse(output, targ): return (output.squeeze(-1) - targ).pow(2).mean()

This is the last basic step for a simple neural net model. So let's integrate it and see how well it can learn.

In [65]:
y_train,y_valid = y_train.float(),y_valid.float()

In [66]:
preds = model(x_train)

preds.shape

torch.Size([60000, 1])

In [67]:
mse(preds, y_train)

tensor(23.1698)